# MODEL TRAINING SETUP

In [30]:
import requests
from pydub import AudioSegment
import os
import pandas as pd
from joblib import Parallel, delayed
# from multiprocessing import Manager, Lock
from tqdm.auto import tqdm
import keras
from keras import layers

In [2]:
import tensorflow as tf
musicnn = tf.keras.models.load_model('./musicnn_keras/keras_checkpoints/MTT_musicnn.h5')
from musicnn_keras.extractor import extractor

/Users/lilhoodie/Libraries/anaconda3/envs/musicnn_env/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [16]:
def get_embedding_musicnn(filename):
    taggram, tags, features = extractor(filename, model='MTT_musicnn', extract_features = True)
    return taggram, tags, features

In [45]:
directory_path = '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio'
file_names = [directory_path + '/' + file for file in os.listdir(directory_path) if file.endswith('.mp3')]
print(file_names[0])

/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/746.mp3


In [46]:
_, _, features = get_embedding_musicnn(file_names[0])
print(features['penultimate'])

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. [[ 0.29161263  0.04805697  0.9511344  ... -0.02580383  0.6288585
   0.5623342 ]
 [-0.07995161  0.22223532  0.88537437 ...  0.12905498  0.8084418
   0.27999815]
 [ 0.12803604  0.7099006   0.48177063 ...  0.2515345   0.8178722
   0.42031375]
 ...
 [ 1.008759    0.43284023 -0.09226304 ... -0.02580383  0.8939617
  -0.1534048 ]
 [ 1.1277146   0.08653146  0.3591339  ...  0.45214477  0.82271177
  -0.1534048 ]
 [ 0.2842933   0.47784856  0.34291032 ... -0.02580383  0.76792645
  -0.09152679]]


In [18]:
print(len(file_names))

1801


In [ ]:
import time

In [47]:
batch_size = 32
batches = [file_names[i:i+batch_size] for i in range(0, len(file_names), batch_size)]
print(batches[0])


['/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/746.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1588.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1563.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1205.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/791.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/949.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1211.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1577.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/2056.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/1239.mp3', '/Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/D

In [50]:
def batch_compute(files):
    results = []
    for file in files:
        try:
            taggram, tags, features = get_embedding_musicnn(file)
            results.append({
                'filename': file,
                'taggram': taggram,
                'tags': tags,
                'features': features,
                # 'embeddings': features['penultimate']
            })
        except Exception as e:
            print(f'{file} errored')
            results.append({'filename': file, 'error': str(e)})
       
    return results
    

full_rows = Parallel(n_jobs=4)(
    delayed(batch_compute)(batch) for batch in tqdm(batches))

full_rows = [file for sublist in full_rows for file in sublist]
musicnn_df = pd.DataFrame(full_rows)

/Users/lilhoodie/Libraries/anaconda3/envs/musicnn_env/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/Users/lilhoodie/Libraries/anaconda3/envs/musicnn_env/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename
/Users/lilhoodie/Libraries/anaconda3/envs/musicnn_env/lib/python3.10/site-packages/librosa/util/files.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. 

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computin

...

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. /Users/lilhoodie/Documents/Personal Projects/SongColor/DEAM/DEAM_audio/MEMD_audio/245.mp3 errored
/Users/lilhoodie/Documents/Persona

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computin

Note: Illegal Audio-MPEG-Header 0x00000000 at offset 124.
Note: Trying to resync...
Note: Hit end of (available) data during resync.


Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. Computin

KeyboardInterrupt: 

# MODEL IMPLEMENTATION

In [ ]:
rnn_model = keras.Sequential()
rnn_model.add(layers.Dense(100))
rnn_model.add(layers.SimpleRNN)
rnn_model.add(layers.Dense(100))

# POST TRAINING IMPLEMENTATION

In [ ]:
def get_embedding_musicnn(song, artist):
    query = song + " " + artist
    # print(query)
    url = f"https://itunes.apple.com/search?term={query}&media=music&entity=musicTrack"

    response = requests.get(url)
    data = response.json()
    preview_url = data['results'][0]['previewUrl']
    # print("Preview URL:", preview_url)

    audio = requests.get(preview_url)
    filename = "../song_clips/"+query + "_preview.wav"

    with open(filename, "wb") as f:
        f.write(audio.content)

    song = AudioSegment.from_file(filename)
    song.export(filename, format="wav")

    taggram, tags, features = extractor(filename, model='MTT_musicnn', extract_features = True)
    # print(list(features.keys()))  
    return features['penultimate']
    

In [5]:
print(get_embedding_musicnn('heart of a woman', 'summer walker'))

Computing spectrogram (w/ librosa) and tags (w/ tensorflow).. WARNING:tensorflow:5 out of the last 5 calls to <function Model.make_predict_function.<locals>.predict_function at 0x317639900> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.
[[-0.07995161  0.19350356  2.0262744  ... -0.02580383  0.87304586
   1.3869632 ]
 [-0.07995161  0.20222324  1.4454312  ... -0.02580383  0.60905164
   0.9412708 ]
 [-0.07995161  0.4415718   2.785294   ... -0.02580383  0.810291
   1.4257473 ]
 .